<a href="https://colab.research.google.com/github/gmauricio-toledo/tda/blob/main/Tareas/Tarea%2002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parte 1: Sistema de recomendación basado en clustering

Realizaremos clustering a un conjunto de datos de canciones de spotify ([más información](https://www.kaggle.com/datasets/joebeachcapital/30000-spotify-songs)). Luego crearemos un sistema de recomendación muy básico para descubrir posibles artistas similares a una lista predeterminada.

Descargamos el conjunto de datos

In [ ]:
!gdown 1C1ORWQbdr3b6L7gd7PODXPmo0SBLyov9

Downloading...
From: https://drive.google.com/uc?id=1C1ORWQbdr3b6L7gd7PODXPmo0SBLyov9
To: /content/spotify_30000.csv
100% 7.97M/7.97M [00:00<00:00, 53.9MB/s]


Lo leemos y quitamos algunas columnas no necesarias

In [ ]:
import pandas as pd

songs_df = pd.read_csv('spotify_30000.csv')
songs_df.drop(columns=['track_id',
                       'track_album_id',
                       'track_album_name',
                       'playlist_name',
                       'playlist_id',
                       'playlist_genre',
                       'playlist_subgenre'], inplace=True)
songs_df

,track_name,track_artist,track_popularity,track_album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2019-06-14,0.748,0.916,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754
1,Memories - Dillon Francis Remix,Maroon 5,67,2019-12-13,0.726,0.815,11,-4.969,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600
2,All the Time - Don Diablo Remix,Zara Larsson,70,2019-07-05,0.675,0.931,1,-3.432,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,2019-07-19,0.718,0.930,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,2019-03-05,0.650,0.833,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,42,2014-04-28,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,2013-03-08,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120
32830,Sweet Surrender - Radio Edit,Starkillers,14,2014-04-21,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112
32831,Only For You - Maor Levi Remix,Mat Zo,15,2014-01-01,0.626,0.888,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432


Damos formato al año

In [ ]:
def convert_date_to_year(date):
  return int(date.split('-')[0])

songs_df['track_album_release_date'] = songs_df['track_album_release_date'].apply(convert_date_to_year)
songs_df

,track_name,track_artist,track_popularity,track_album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2019,0.748,0.916,6,-2.634,1,0.0583,0.102000,0.000000,0.0653,0.5180,122.036,194754
1,Memories - Dillon Francis Remix,Maroon 5,67,2019,0.726,0.815,11,-4.969,1,0.0373,0.072400,0.004210,0.3570,0.6930,99.972,162600
2,All the Time - Don Diablo Remix,Zara Larsson,70,2019,0.675,0.931,1,-3.432,0,0.0742,0.079400,0.000023,0.1100,0.6130,124.008,176616
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,2019,0.718,0.930,7,-3.778,1,0.1020,0.028700,0.000009,0.2040,0.2770,121.956,169093
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,2019,0.650,0.833,1,-4.672,1,0.0359,0.080300,0.000000,0.0833,0.7250,123.976,189052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,42,2014,0.428,0.922,2,-1.814,1,0.0936,0.076600,0.000000,0.0668,0.2100,128.170,204375
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20,2013,0.522,0.786,0,-4.462,1,0.0420,0.001710,0.004270,0.3750,0.4000,128.041,353120
32830,Sweet Surrender - Radio Edit,Starkillers,14,2014,0.529,0.821,6,-4.899,0,0.0481,0.108000,0.000001,0.1500,0.4360,127.989,210112
32831,Only For You - Maor Levi Remix,Mat Zo,15,2014,0.626,0.888,2,-3.361,1,0.1090,0.007920,0.127000,0.3430,0.3080,128.008,367432


Define las variables que desecharás para hacer clustering

In [ ]:
columns_to_drop = ['track_name',
                   'track_artist',
                   ...
                    ]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Conjunto de datos:
X = songs_df.drop(columns=columns_to_drop).values

# Reescalamos las variables al hipercubo unitario:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Visualizamos:
pd.DataFrame(X, columns=songs_df.drop(columns=['track_name', 'track_artist']).columns)

,track_popularity,track_album_release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.66,0.984127,0.760936,0.915985,0.545455,0.918090,1.0,0.063508,0.102616,0.000000,0.065562,0.522704,0.509673,0.371254
1,0.67,0.984127,0.738555,0.814968,1.000000,0.869162,1.0,0.040632,0.072837,0.004235,0.358434,0.699294,0.417524,0.308674
2,0.70,0.984127,0.686673,0.930988,0.090909,0.901368,0.0,0.080828,0.079879,0.000023,0.110442,0.618567,0.517908,0.335953
3,0.60,0.984127,0.730417,0.929988,0.636364,0.894118,1.0,0.111111,0.028873,0.000009,0.204819,0.279516,0.509338,0.321311
4,0.69,0.984127,0.661241,0.832971,0.090909,0.875385,1.0,0.039107,0.080785,0.000000,0.083635,0.731584,0.517775,0.360156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,0.42,0.904762,0.435402,0.921986,0.181818,0.935272,1.0,0.101961,0.077062,0.000000,0.067068,0.211907,0.535291,0.389979
32829,0.20,0.888889,0.531027,0.785963,0.000000,0.879785,1.0,0.045752,0.001720,0.004296,0.376506,0.403633,0.534752,0.679473
32830,0.14,0.904762,0.538149,0.820969,0.545455,0.870628,0.0,0.052397,0.108652,0.000001,0.150602,0.439960,0.534535,0.401144
32831,0.15,0.904762,0.636826,0.887980,0.181818,0.902856,1.0,0.118736,0.007968,0.127767,0.344378,0.310797,0.534614,0.707328


Aplica clustering, determina el número de clusters graficando el valor de silueta vs el número de clusters

In [ ]:
#----- APLICA CLUSTERING -----
...

clusters = kmeans.labels_
#----------------------------------

,track_name,track_artist,cluster
0,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,4
1,Memories - Dillon Francis Remix,Maroon 5,4
2,All the Time - Don Diablo Remix,Zara Larsson,14
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,0
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,9
...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,18
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,2
32830,Sweet Surrender - Radio Edit,Starkillers,3
32831,Only For You - Maor Levi Remix,Mat Zo,2


Creamos un dataframe con la información de artistas, canciones y cluster

In [ ]:
clusters_df = songs_df[['track_name', 'track_artist']].copy()
clusters_df['cluster'] = clusters
clusters_df

En esta parte escogerás algunos artistas que prefieras para obtener recomendaciones similares a ellos

Dado que son muchos:

In [ ]:
import numpy as np

artistas = clusters_df['track_artist'].unique()
print(f"Número de artistas: {len(artistas)}")

Número de artistas: 10693


Una opción es imprimir algunos aleatorios para explorarlos

array(['Nate VanDeusen', 'eSQUIRE', 'Healy', 'Bananarama', 'Josh Rouse',
       'UZ', 'ENCR', 'Thom Yorke', 'Dan Bravo', 'Mile Twelve',
       "Raja-Nee'", 'Tracie Spencer', 'Lin-Manuel Miranda', 'Klingande',
       'Fotty Seven'], dtype=object)

También puedes probar a buscar directamente en la lista

In [ ]:
'Yes' in artistas

Escoge varios:

In [ ]:
favoritos = ['Yes',
             ...
             ]

🔴 Descifrar cuál es la estrategia para hacer la recomendación:

In [ ]:
clusters_favoritos = clusters_df[clusters_df['track_artist'].isin(favoritos)]['cluster'].unique()
recomendaciones = clusters_df[clusters_df['cluster'].isin(clusters_favoritos)]['track_artist'].unique()
print(f"Número de recomendaciones: {len(recomendaciones)}")
print(f"Recomendaciones:\n{recomendaciones}")

Número de recomendaciones: 6242
Recomendaciones:
['Lewis Capaldi' 'Shawn Mendes' 'Loud Luxury' ... 'Audien'
 'Tegan and Sara' 'Mat Zo']


Imprime o explora las recomendaciones:

array(['Maren Morris', 'Falling Decibyls', 'Cypress Hill',
       'LaMorris Williams', 'Camille', 'Zwette', 'Nena', 'Fayn',
       'Skrillex', 'Rubén Blades', 'Travis', 'Renato S', 'Hey Violet',
       'Jermaine Hardsoul', 'Hp Boyz'], dtype=object)

🔴 Repite el proceso de clustering hasta que obtengas resultados razonablemente satisfactorios

🔴 Describe una posible mejora para este sistema de recomendación

# Parte 2

1. Dados puntos $a_0,...,a_n \in \mathbb{R}^n$ geométricamente independientes, $\sigma$ el $n$ simplejo generado por los puntos, y un punto $p\in\sigma$, demuestra que existen únicos $t_0,...,t_n\in \mathbb{R}$ tales que $t_i\geq 0$, $\sum t_i = 1$ y

$$\sum_{i=0}^n t_i a_i = p$$

2. Considera un triángulo equilatero en $\mathbb{R}^2$, encuentra las coordenadas baricéntricas del baricentro.
3. Considera el triángulo con vértices $a_0=(-1,-1)$, $a_1=(1,0)$ y $a_2=(0,1)$. Encuentra las coordenadas baricéntricas del origen.
4. Dados puntos $a_0,...,a_n \in \mathbb{R}^n$ geométricamente independientes, sea $\sigma$ el $n$ simplejo generado por los puntos. Demuestra que $\sigma$ es convexo.
5. Demuestra que el simplex generado por 4 puntos geométricamente independientes es un tetraedro *relleno*.